Exercise 1

In [ ]:
## A perceptron is a building block of a neural network that is an abstraction of a biological neuron, which is a simple but powerful comutational unit that makes up the simplet neural architecture.It was named by Frank Rosenblatt in 1957. 

Exercise 2

In [ ]:
## There is the perceptron, which is a single layer of input linear neurons that are following by an output unit based on a function. Then there is the multilayer perceptron. This is adding one non-linear layer between the input and the outpuds which would lead to a highly non-linear combination.

Exercise 3

In [ ]:
## A hard margin in a Support Vector Machine is very rigid in classification and tries to perform extremely well in the training set.

Exercise 4

In [ ]:
## d. None of the above

Exercise 5

In [ ]:
## d. a and c

Exercise 6

In [1]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import tensorflow as tf

## Defining the s3 bucket
s3 = boto3.resource('s3')
bucket_name = 'mckayla-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Removing the observations with N/As
heart = heart.dropna()

In [3]:
## Defining the input and target
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)

In [4]:
## Min-Max transformation
scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [5]:
## Defining the model
md1 = tf.keras.models.Sequential([
      tf.keras.layers.Dense(4, input_dim = 2, activation = 'tanh'),
      tf.keras.layers.Dense(2, activation = 'softmax'),
      tf.keras.layers.Dense(1)
])

md1.compile(optimizer = 'sgd', metrics = ['accuracy'])
history = md1.fit(X_train, Y_train, epochs = 100, batch_size = 500, validation_data = (X_test, Y_test))

Epoch 1/100


ValueError: in user code:

    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:801 train_function  *
        return step_function(self, iterator)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:791 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/

In [ ]:
md1.evaluate(X_test, Y_test)

In [ ]:
## Predicting on test
logit_pred = logit_md.predict_proba(X_test)[:,1]

## Changing likelihoods to labels
logit_labels = np.where(logit_pred < 0.1, 0, 1)

## Computing the recall-score
logit_recall = recall_score(Y_test, logit_labels)
print('The recall score for the logistic model is', logit_recall)

In [20]:
## Defining the model
md2 = tf.keras.models.Sequential([
      tf.keras.layers.Dense(4, input_dim = 2, activation = 'relu'),
      tf.keras.layers.Dense(2, activation = 'softmax'),
      tf.keras.layers.Dense(1)
])

md2.compile(optimizer = 'sgd', metrics = ['accuracy'])
history = md2.fit(X_train,tf.keras.utils.to_categorical(Y_train, num_classes = 2), epochs = 100, batch_size = 500, validation_data = (X_test, tf.keras.utils.to_categorical(Y_test, num_classes = 2)))

Epoch 1/100


ValueError: in user code:

    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:801 train_function  *
        return step_function(self, iterator)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py:791 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:1211 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/distribute/distribute_lib.py:2585 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/tensorflow/python/

In [ ]:
md2.evaluate(X_test, Y_test)

In [ ]:
## Predicting on test
logit_pred = logit_md.predict_proba(X_test)[:,1]

## Changing likelihoods to labels
logit_labels = np.where(logit_pred < 0.1, 0, 1)

## Computing the recall-score
logit_recall = recall_score(Y_test, logit_labels)
print('The recall score for the logistic model is', logit_recall)